# [05] 모델 만들기4 - DetectionNet

이전 실습에서 만들었던 `BaseConv`, `AuxConv`, `PredConv`를 모두 함께 사용하여 하나의 `nn.Module` Class로 구성해 봅시다. materials.det_modules에 작성되어 있는 해당 모듈들의 예시입니다. 이번 실습에서는 아래의 모듈들을 활용해 `DetectionNet`을 만들어 보겠습니다.

In [1]:
import torch
from materials.det_modules import BaseConvolution, AuxConvolution, PredictionConvolution

baseconv = BaseConvolution()
auxconv = AuxConvolution()
predconv = PredictionConvolution()

sample = torch.randn(4, 3, 300, 300)
mid, end = baseconv(sample)
features_a, features_b, features_c , features_d = auxconv(end)
locs, cls_scores = predconv(mid, end, features_a, features_b, features_c, features_d)

print('\nmid shape {}'.format(mid.shape))
print('end shape {}'.format(end.shape))
print('map a shape : {}'.format(features_a.shape))
print('map b shape : {}'.format(features_b.shape))
print('map c shape : {}'.format(features_c.shape))
print('map d shape : {}'.format(features_d.shape))

print('\nLocation 예측의 Shape: {}'.format(locs.shape))
print('Clsss Score 예측의 Shape: {}'.format(cls_scores.shape))

Loaded pretrained weights for efficientnet-b0

mid shape torch.Size([4, 112, 18, 18])
end shape torch.Size([4, 1280, 9, 9])
map a shape : torch.Size([4, 512, 9, 9])
map b shape : torch.Size([4, 256, 5, 5])
map c shape : torch.Size([4, 256, 3, 3])
map d shape : torch.Size([4, 256, 1, 1])

Location 예측의 Shape: torch.Size([4, 3106, 4])
Clsss Score 예측의 Shape: torch.Size([4, 3106, 20])


-------------------
## [Task 1] `DetectionNet` 완성하기

`DetectionNet`을 완성해 봅시다. 3가지의 모듈을 차례대로 거쳐 출력값을 얻도록 구성합니다. 다만, `mid` featuremap에 대해 normalize한 다음 채널별로 값을 하나씩 곱해 rescale해주고, 이 값들이 학습되도록 할 것입니다.

- `__init__()` 함수를 완성합니다. `mid`에 대해 학습되는 Parameter를 곱합니다.

- `forward` 함수를 완성합니다. normalize와 rescale의 과정이 포함되어야 합니다.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from materials.det_modules import BaseConvolution, AuxConvolution, PredictionConvolution

class DetectionNet(nn.Module):
    def __init__(self, n_classes=20, unfreeze_keys=['15', 'head', 'bn1']):
        super(DetectionNet, self).__init__()
        self.n_classes = n_classes
        
        # [ToDo]: BaseConv를 구성합니다.
        self.base = ???????????????????
        
        # [ToDo]: AuxConv를 구성합니다.
        self.aux = ???????????????????
        
        # [ToDo]: PredConv를 구성합니다.
        self.pred = ???????????????????
        
        # [ToDo]: mid feature의 각 channel에 곱해줄 parameter를 선언합니다.
        self.rescale_mid = nn.Parameter(torch.FloatTensor(?, ?, ?, ?))
        
        # rescale parameter를 initialize합니다.
        nn.init.constant_(self.rescale_mid, 5)
        
    
    def forward(self, images):
        # [ToDo]: baseconv를 수행합니다.
        mid, end = ???????????????????
        
        # mid featuremap에 대해 normalize와 rescale을 합니다.
        mid = F.normalize(mid)
        mid = self.rescale_mid * mid
        
        # [ToDo]: auxconv를 수행합니다.
        a, b, c, d = ???????????????????
        
        # [ToDo]: predconv를 수행합니다.
        locs, cls_scores = ???????????????????
        
        return locs, cls_scores

In [3]:
sample = torch.randn(4, 3, 300, 300)
net = DetectionNet(n_classes=20, unfreeze_keys=['15', 'head', 'bn1'])
locs, cls_scores = net(sample)

print('\nLocation 예측의 Shape: {}'.format(locs.shape))
print('Clsss Score 예측의 Shape: {}'.format(cls_scores.shape))

Loaded pretrained weights for efficientnet-b0

Location 예측의 Shape: torch.Size([4, 3106, 4])
Clsss Score 예측의 Shape: torch.Size([4, 3106, 20])


---------
### <생각해 봅시다>

- normalize와 rescale의 과정은 왜 필요할까요?
- rescale parameter들은 어떻게 변할까요?

------------